<a href="https://colab.research.google.com/github/SijiJ/SONiC/blob/master/fashion_assgn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16512176268326831739, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14509932544
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15284631857284812524
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [3]:
!cat /proc/meminfo

MemTotal:       13305360 kB
MemFree:         9445844 kB
MemAvailable:   12041764 kB
Buffers:           86816 kB
Cached:          2509532 kB
SwapCached:            0 kB
Active:          1283456 kB
Inactive:        2177060 kB
Active(anon):     683128 kB
Inactive(anon):    10668 kB
Active(file):     600328 kB
Inactive(file):  2166392 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               148 kB
Writeback:             0 kB
AnonPages:        864228 kB
Mapped:           599592 kB
Shmem:             11408 kB
KReclaimable:     141556 kB
Slab:             198088 kB
SReclaimable:     141556 kB
SUnreclaim:        56532 kB
KernelStack:        5168 kB
PageTables:         8728 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6652680 kB
Committed_AS:    3861644 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       51308 kB
VmallocChunk:          0 kB
Percpu:          

In [18]:
import tensorflow as tf

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

import random
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [22]:
random.seed(42)         # Initialize the random number generator.
np.random.seed(42)      # With the seed reset, the same set of numbers will appear every time. 
#tf.set_random_seed(42)  # sets the graph-level random seed
tf.random.set_seed(42)

# **Data Visualization and augmentation**

In [5]:
!git clone https://github.com/SijiJ/fashion-mnist.git

Cloning into 'fashion-mnist'...
remote: Enumerating objects: 758, done.
remote: Total 758 (delta 0), reused 0 (delta 0), pack-reused 758
Receiving objects: 100% (758/758), 105.84 MiB | 47.72 MiB/s, done.
Resolving deltas: 100% (444/444), done.


In [19]:
!ls fashion-mnist/data/fashion

t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [52]:
!ls fashion-mnist/

app.py		 data	     MAINTAINERS      requirements.txt
benchmark	 doc	     README.ja.md     static
configs.py	 Dockerfile  README.md	      utils
CONTRIBUTING.md  LICENSE     README.zh-CN.md  visualization


In [ ]:
!cat fashion-mnist/utils/helper.py

In [55]:
import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('fashion-mnist')
from utils import mnist_reader
Xtrain, Ytrain = mnist_reader.load_mnist('fashion-mnist/data/fashion', kind='train')
Xtest, Ytest = mnist_reader.load_mnist('fashion-mnist/data/fashion', kind='t10k')


In [56]:
print(Xtrain.shape)
print(Xtest.shape)
print(Ytrain.shape)
print(Ytest.shape)

(60000, 784)
(10000, 784)
(60000,)
(10000,)


In [57]:
type(Ytrain)
Ytrain[:10]

array([9, 0, 0, 3, 0, 2, 7, 2, 5, 5], dtype=uint8)

In [58]:
# print a sample data
labels = ['t_shirt_top', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle_boots']
Ytrain_str = np.array([labels[j] for j in Ytrain])
np.savetxt('Xtest.tsv', Xtrain, fmt='%.6e', delimiter='\t')
np.savetxt('Ytest.tsv', Ytrain_str, fmt='%s')

print('Xtrain \n', Xtrain[:10][:10])
print('Xtest \n', Xtest[:10,:10])
print('Ytrain \n', Ytrain[:10,])
print('Ytest \n', Ytest[:10,])

Xtrain 
 [[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ... 70  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
Xtest 
 [[ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  0 21]
 [ 0  0  0  2  0  1  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0  0  2  3]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]
Ytrain 
 [9 0 0 3 0 2 7 2 5 5]
Ytest 
 [9 2 1 1 6 1 4 6 5 7]


In [59]:
!cat Xtest.tsv |head -10

0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00

In [60]:
!cat Ytest.tsv |head -10

ankle_boots
t_shirt_top
t_shirt_top
dress
t_shirt_top
pullover
sneaker
pullover
sandal
sandal


In [61]:
from helper import get_sprite_image
from configs import DATA_DIR, LOG_DIR, VIS_DIR

In [63]:
plt.imsave('zalando-mnist-sprite.png', get_sprite_image(Xtest), cmap='gray')

In [64]:
!ls

fashion-mnist  sample_data  Xtest.tsv  Ytest.tsv  zalando-mnist-sprite.png


In [ ]:
from IPython.display import Image
Image('zalando-mnist-sprite.png')

In [71]:
print(type(Xtrain))
print(type(Ytrain_str))
XYtrain = np.c_[ Xtrain, Ytrain_str ]    
print(type(Xtrain))
print(XYtrain.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(60000, 785)


In [73]:
aboots = XYtrain[XYtrain[:, -1] == "ankle_boots"]

In [74]:
print(aboots.shape)

(6000, 785)


In [76]:
print(aboots[:, -1])

['ankle_boots' 'ankle_boots' 'ankle_boots' ... 'ankle_boots' 'ankle_boots'
 'ankle_boots']
